In [1]:
import numpy as np
import pandas as pd
import glob
import os
#for dirname, _, filenames in os.walk('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.optimizers import Adam, SGD
import pandas as pd

In [2]:
Mild_OVS = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/MildDemented/*.*')
Moderate_OVS = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/ModerateDemented/*.*')
NonDemented_OVS = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/NonDemented/*.*')
VeryMild_OVS = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/VeryMildDemented/*.*')


data_OVS=[]
labels_OVS = []

for i in NonDemented_OVS:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224))
    image=np.array(image)
    data_OVS.append(image)
    labels_OVS.append(0)

for i in VeryMild_OVS:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224))
    image=np.array(image)
    data_OVS.append(image)
    labels_OVS.append(1)

for i in Mild_OVS:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224))
    image=np.array(image)
    data_OVS.append(image)
    labels_OVS.append(2)

for i in Moderate_OVS:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224))
    image=np.array(image)
    data_OVS.append(image)
    labels_OVS.append(3)

In [3]:
data_OVS = np.array(data_OVS)
labels_OVS = np.array(labels_OVS)

from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
data_OVS,labels_OVS = sm.fit_resample(data_OVS.reshape(-1, 224 * 224 * 3), labels_OVS)

data_OVS = data_OVS.reshape(-1, 224, 224, 3)

print(data_OVS.shape, labels_OVS.shape)
print(len(data_OVS))

(10240, 224, 224, 3) (10240,)
10240


In [4]:
import glob
Mild_test = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/MildDemented/*.*')
Moderate_test = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/ModerateDemented/*.*')
NonDemented_test = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/NonDemented/*.*')
VeryMild_test = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/VeryMildDemented/*.*')


data_test = []
labels_test  = []

for i in NonDemented_test:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),grayscale=False)
    image=np.array(image)
    data_test.append(image)
    labels_test.append(0)

for i in VeryMild_test:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),grayscale=False)
    image=np.array(image)
    data_test.append(image)
    labels_test.append(1)

for i in Mild_test:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),grayscale=False)
    image=np.array(image)
    data_test.append(image)
    labels_test.append(2)

for i in Moderate_test:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),grayscale=False)
    image=np.array(image)
    data_test.append(image)
    labels_test.append(3)

In [5]:
data_test = np.array(data_test)
labels_test = np.array(labels_test)

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255,
                            rotation_range=45,
                            width_shift_range=.5,
                            height_shift_range=.5,
                            horizontal_flip=False)

train_data_gen = datagen.flow(
    x=data_OVS,
    y=labels_OVS,
    batch_size=64,
)

In [7]:
test_data_gen = datagen.flow(
    x=data_test,
    y=labels_test
)

In [8]:
from keras.utils import to_categorical

train_data_gen.y = to_categorical(train_data_gen.y, num_classes=4)
test_data_gen.y = to_categorical(test_data_gen.y, num_classes=4)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [10]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

In [11]:
from keras.callbacks import Callback
import numpy as np
from sklearn.metrics import f1_score

class MetricsCallback(Callback):
    def __init__(self, validation_data):
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        x_val, y_val = self.validation_data
        y_pred = np.argmax(self.model.predict(x_val), axis=1)

        true_positives = np.sum(np.logical_and(y_val == 1, y_pred == 1))
        false_positives = np.sum(np.logical_and(y_val == 0, y_pred == 1))
        false_negatives = np.sum(np.logical_and(y_val == 1, y_pred == 0))

        precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        print(f'\nPrecision: {precision:.4f} - Recall: {recall:.4f} - F1 Score: {f1:.4f}')

In [12]:
import keras
import tensorflow_addons as tfa
METRICS = [keras.metrics.CategoricalAccuracy(name='accuracy'),
           keras.metrics.Precision(name='precision'),
           keras.metrics.Recall(name='recall'),
           keras.metrics.AUC(name='auc'),
           tfa.metrics.F1Score(num_classes=4),
           #tfa.metrics.MultiLabelConfusionMatrix(num_classes=4, name='confusion_matrix')
          ]

/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [13]:
# Compile the model
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Train the model
#history = model.fit(train_data_gen.x,train_data_gen.y ,batch_size=128, epochs=100,validation_data=test_data_gen, callbacks=[MetricsCallback((test_data_gen.x, test_data_gen.y))])

In [14]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=METRICS)

# Train the model with the metrics callback
history = model.fit(train_data_gen.x, train_data_gen.y, batch_size=128, epochs=30, validation_data=(test_data_gen.x, test_data_gen.y))


Epoch 1/30
80/80 [==============================] - 676s 8s/step - loss: 43.4948 - accuracy: 0.4118 - precision: 0.4536 - recall: 0.1698 - auc: 0.6818 - f1_score: 0.3875 - val_loss: 1.0221 - val_accuracy: 0.5551 - val_precision: 0.5716 - val_recall: 0.4652 - val_auc: 0.8098 - val_f1_score: 0.3012
Epoch 2/30
80/80 [==============================] - 678s 8s/step - loss: 0.9726 - accuracy: 0.5354 - precision: 0.7128 - recall: 0.3433 - auc: 0.8208 - f1_score: 0.5217 - val_loss: 0.8889 - val_accuracy: 0.5786 - val_precision: 0.7504 - val_recall: 0.3268 - val_auc: 0.8515 - val_f1_score: 0.3173
Epoch 3/30
80/80 [==============================] - 658s 8s/step - loss: 0.8770 - accuracy: 0.5705 - precision: 0.7423 - recall: 0.4188 - auc: 0.8552 - f1_score: 0.5602 - val_loss: 0.9219 - val_accuracy: 0.5434 - val_precision: 0.6253 - val_recall: 0.4214 - val_auc: 0.8451 - val_f1_score: 0.2762
Epoch 4/30
80/80 [==============================] - 658s 8s/step - loss: 0.8119 - accuracy: 0.6059 - precisi

In [15]:
import seaborn as sns
from keras.callbacks import Callback
from sklearn.metrics import classification_report, confusion_matrix
#final_predictions = np.argmax(model.predict(test_data_gen.x), axis=1)
true_labels = np.argmax(test_data_gen.y, axis=1)

final_predictions = np.argmax(model.predict(test_data_gen.x), axis=1)
final_predictions_onehot = to_categorical(final_predictions, num_classes=4)

# Compute the confusion matrix
final_cm = confusion_matrix(test_data_gen.y.argmax(axis=1), final_predictions_onehot.argmax(axis=1))

# Display the confusion matrix
print("\nFinal Confusion Matrix:")
print(final_cm)

# Visualize the confusion matrix using seaborn
#plt.figure(figsize=(8, 6))
#sns.heatmap(final_cm, annot=True, fmt="d", cmap="Blues", xticklabels=[f'Class {i}' for i in range(metrics_callback.num_classes)], yticklabels=[f'Class {i}' for i in range(metrics_callback.num_classes)])
#plt.xlabel('Predicted')
#plt.ylabel('True')
#plt.title('Final Confusion Matrix')
#plt.show()

40/40 [==============================] - 23s 563ms/step

Final Confusion Matrix:
[[245 297  98   0]
 [ 53 264 131   0]
 [ 16  66  97   0]
 [  2   2   6   2]]


In [16]:
from sklearn.metrics import classification_report

print(classification_report(test_data_gen.y.argmax(axis=1), final_predictions_onehot.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.78      0.38      0.51       640
           1       0.42      0.59      0.49       448
           2       0.29      0.54      0.38       179
           3       1.00      0.17      0.29        12

    accuracy                           0.48      1279
   macro avg       0.62      0.42      0.42      1279
weighted avg       0.59      0.48      0.48      1279



In [17]:
model.save('my_modelO.h5')

In [18]:
#model.save('my_model30.pkl')